In [6]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [7]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Shenjiamen,29.9576,122.2980,70.07,76,98,16.71,CN,2022-10-14 04:21:03
1,1,Sitka,57.0531,-135.3300,51.69,71,75,9.22,US,2022-10-14 04:21:04
2,2,Dhidhdhoo,6.8833,73.1000,81.12,76,99,14.25,MV,2022-10-14 04:16:27
3,3,Benjamin Hill,30.1667,-111.1000,77.79,41,98,3.31,MX,2022-10-14 04:21:04
4,4,Kasongo,-4.4500,26.6667,67.14,85,71,0.89,CD,2022-10-14 04:21:05


In [8]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [9]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [10]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# heat_layer = gmaps.heatmap_layer(locations, weights=temps)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []

for temp in max_temp:
    temps.append(max(temp, 0))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Dhidhdhoo,6.8833,73.1000,81.12,76,99,14.25,MV,2022-10-14 04:16:27
3,3,Benjamin Hill,30.1667,-111.1000,77.79,41,98,3.31,MX,2022-10-14 04:21:04
15,15,Surt,31.2089,16.5887,79.75,44,97,19.15,LY,2022-10-14 04:21:11
25,25,Sao Filipe,14.8961,-24.4956,78.26,81,85,9.78,CV,2022-10-14 04:21:15
33,33,Ternate,0.8000,127.4000,85.03,71,38,10.02,ID,2022-10-14 04:21:18
40,40,Araouane,18.9048,-3.5265,87.60,13,28,7.99,ML,2022-10-14 04:21:21
43,43,Salinopolis,-0.6136,-47.3561,80.31,82,33,11.72,BR,2022-10-14 04:21:22
44,44,Berbera,10.4396,45.0143,83.07,67,1,2.66,SO,2022-10-14 04:21:22
47,47,Mahebourg,-20.4081,57.7000,75.25,68,40,14.97,MU,2022-10-14 04:14:52
53,53,Nhulunbuy,-12.2333,136.7667,86.61,66,40,17.27,AU,2022-10-14 04:22:26


In [15]:
preferred_cities_df.count()

City_ID       157
City          157
Lat           157
Lng           157
Max Temp      157
Humidity      157
Cloudiness    157
Wind Speed    157
Country       157
Date          157
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Dhidhdhoo,MV,81.12,6.8833,73.1000,
3,Benjamin Hill,MX,77.79,30.1667,-111.1000,
15,Surt,LY,79.75,31.2089,16.5887,
25,Sao Filipe,CV,78.26,14.8961,-24.4956,
33,Ternate,ID,85.03,0.8000,127.4000,
40,Araouane,ML,87.60,18.9048,-3.5265,
43,Salinopolis,BR,80.31,-0.6136,-47.3561,
44,Berbera,SO,83.07,10.4396,45.0143,
47,Mahebourg,MU,75.25,-20.4081,57.7000,
53,Nhulunbuy,AU,86.61,-12.2333,136.7667,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# # Dependencies and Setup
# import requests
# import gmaps

# # Import API key
# from config import g_key

# # Set the parameters to search for a hotel in Paris.
# params = {
#     "radius": 5000,
#     "types": "lodging",
#     "key": g_key,
#     "location": "48.8566, 2.3522"}
# # Use base URL to search for hotels in Paris.
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# # Make request and get the JSON data from the search.
# hotels = requests.get(base_url, params=params).json()

# hotels

# len(hotels["results"])

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
#     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Dhidhdhoo,MV,81.12,6.8833,73.1000,Dhidhoo beach
3,Benjamin Hill,MX,77.79,30.1667,-111.1000,Hotel Benjamín Hill
15,Surt,LY,79.75,31.2089,16.5887,فندق المدينة
25,Sao Filipe,CV,78.26,14.8961,-24.4956,Hotel Xaguate
33,Ternate,ID,85.03,0.8000,127.4000,Sahid Bela Ternate Hotel
40,Araouane,ML,87.60,18.9048,-3.5265,
43,Salinopolis,BR,80.31,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
44,Berbera,SO,83.07,10.4396,45.0143,IBCO REAL ESTATE PROPERTY
47,Mahebourg,MU,75.25,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
53,Nhulunbuy,AU,86.61,-12.2333,136.7667,East Arnhem Real Estate


In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [22]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))